In [1]:
#STRIVING FOR SIMPLICITY:
#THE ALL CONVOLUTIONAL NET
#https://arxiv.org/pdf/1412.6806.pdf
#With modifications replacing relu with elu
#kernel_initializer used is he_normal
#Using admax instead of stedily decreasing lr_rate with a sgd

In [2]:
#Doing Array operations if needed
import numpy as np

#Making the NN
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

#Loading images from disk
from scipy import misc
#Getting the name of all the files
import glob
#Handling data in df and saving to csv
import pandas as pd

#One hot encoding of labels
from sklearn.preprocessing import LabelBinarizer
#Splitting the data set for train and validation if needed
#You can also use validation split
from sklearn.model_selection import train_test_split
#Handling pickle files
import pickle

Using TensorFlow backend.


In [ ]:
#Load target labels
train_labels=pd.read_csv('dataset/trainLabels.csv')
train_labels.set_index('id', inplace=True)
train_labels=train_labels.to_dict(orient='id')

In [ ]:
#Loading the dataset
x_train=[]
y_train=[]

count=0
for image_path in glob.glob("dataset/train/*.png"):
    count+=1
    if (count%5000==0):
        print(count)
    x_train.append(misc.imread(image_path))
    temp=image_path.split('\\')[-1].split('.')[0]
    label=train_labels[int(temp)]['label']
    y_train.append(label)

In [ ]:
#For Debugging purposes
x_train=np.array(x_train)
print(x_train.shape)

In [ ]:
lb = LabelBinarizer()
y_train=lb.fit_transform(y_train)
print(y_train.shape)

In [ ]:
# save the labelBinarizer to disk
from sklearn.externals import joblib
joblib.dump(lb, 'label.pkl')

In [ ]:
#This can be a faster way to save/load as compared to loading images everytime
file = open('x_train', 'wb')
pickle.dump(x_train,file)
file.close()
file = open('y_train', 'wb')
pickle.dump(y_train,file)
file.close()

In [3]:
#I pickled train dataset
import pickle

with open('x_train', 'rb') as f:
    x_train = pickle.load(f) 

with open('y_train', 'rb') as f:
    y_train = pickle.load(f) 

In [4]:
#Normalizing the images
x_train = x_train.astype('float32')
x_train/= 255

In [5]:
#Train and val split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [6]:
#For debugging puposes
x_train.shape

(42500, 32, 32, 3)

In [ ]:
#Making this generator beforehand
imdgen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False,  # randomly flip images
    shear_range=0.2,       
    zoom_range=0.2,
)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
imdgen.fit(x_train)

# fit the model on the batches generated by datagen.flow()
dgen = imdgen.flow(x_train, y_train, batch_size=128)


In [ ]:
# For reproducibility
np.random.seed(1000)

# Load the dataset
#(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

# Create the model
model = Sequential()
model.add(Dropout(0.2, input_shape=(32,32,3)))
model.add(Conv2D(96, (3,3), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(96, (3,3), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(96, (3,3), strides=(2,2), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Conv2D(192, (3,3), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(192, (3,3), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(192, (3,3), strides=(2,2), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Conv2D(192, (3,3), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(192, (1,1), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(10, (1,1), strides=(1,1), padding='same',activation='elu',kernel_initializer='he_normal'))

model.add(AveragePooling2D(pool_size=(6, 6), strides=None, padding='valid', data_format=None))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
checkpoint = ModelCheckpoint("CIFAR_10_ALL CONVOLUTIONAL_NET.h5", monitor='val_loss', verbose=2, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['categorical_accuracy'])
model.summary()

In [11]:
#Load the best weights if you are going to train model from a previous checkpoint
#model.load_weights('CIFAR_10_ALL CONVOLUTIONAL_NET.h5')
# Train the model
#model.fit_generator(dgen,
#                    steps_per_epoch=x_train.shape[0]/128,
#                    epochs=100,
#                    validation_data=(x_val, y_val),
#                    workers=3,callbacks=callbacks_list)

# Train the model
model.fit(x_train, y_train,batch_size=128,epochs=20,validation_data=(x_val, y_val),callbacks=callbacks_list,shuffle=True)

Epoch 1/100
333/332 [==============================] - 294s - loss: 0.6412 - categorical_accuracy: 0.7775 - val_loss: 0.5391 - val_categorical_accuracy: 0.8237
Epoch 2/100
333/332 [==============================] - 292s - loss: 0.6205 - categorical_accuracy: 0.7844 - val_loss: 0.5711 - val_categorical_accuracy: 0.8145
Epoch 3/100
333/332 [==============================] - 291s - loss: 0.6183 - categorical_accuracy: 0.7854 - val_loss: 0.5542 - val_categorical_accuracy: 0.8183
Epoch 4/100
333/332 [==============================] - 289s - loss: 0.6011 - categorical_accuracy: 0.7911 - val_loss: 0.6070 - val_categorical_accuracy: 0.7989
Epoch 5/100
333/332 [==============================] - 291s - loss: 0.6030 - categorical_accuracy: 0.7893 - val_loss: 0.5175 - val_categorical_accuracy: 0.8247
Epoch 6/100
333/332 [==============================] - 292s - loss: 0.5907 - categorical_accuracy: 0.7957 - val_loss: 0.5754 - val_categorical_accuracy: 0.8109
Epoch 7/100
333/332 [===================

333/332 [==============================] - 292s - loss: 0.4908 - categorical_accuracy: 0.8281 - val_loss: 0.4959 - val_categorical_accuracy: 0.8443
Epoch 55/100
333/332 [==============================] - 292s - loss: 0.4930 - categorical_accuracy: 0.8278 - val_loss: 0.5083 - val_categorical_accuracy: 0.8383
Epoch 56/100
333/332 [==============================] - 293s - loss: 0.4957 - categorical_accuracy: 0.8280 - val_loss: 0.4923 - val_categorical_accuracy: 0.8459
Epoch 57/100
333/332 [==============================] - 292s - loss: 0.4963 - categorical_accuracy: 0.8278 - val_loss: 0.4922 - val_categorical_accuracy: 0.8457
Epoch 58/100
333/332 [==============================] - 292s - loss: 0.4933 - categorical_accuracy: 0.8297 - val_loss: 0.4797 - val_categorical_accuracy: 0.8471
Epoch 59/100
333/332 [==============================] - 293s - loss: 0.4889 - categorical_accuracy: 0.8314 - val_loss: 0.5192 - val_categorical_accuracy: 0.8355
Epoch 60/100
333/332 [=========================

In [12]:
model.load_weights('CIFAR_10_ALL CONVOLUTIONAL_NET.h5')
from sklearn.externals import joblib
lb=joblib.load('label.pkl')

In [13]:
output=pd.DataFrame(columns=['id','label'])
count=0
for image_path in glob.glob("dataset/test/test/*.png"):
    count+=1
    if (count%5000==0):
        print(count)
    img=misc.imread(image_path).astype('float32')
    img/= 255
    img=np.array([img])
    pred=model.predict(img)
    pred=lb.inverse_transform(pred)
    index=image_path.split('\\')[-1].split('.')[0]
    output=output.append({'id':index, 'label':pred[0]}, ignore_index=True)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000


In [14]:
output.to_csv('output_cnc.csv',index=False)